In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
!pip install polars
# !pip install pickle5

In [ ]:
import polars as pl
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

train = pl.read_parquet('/content/drive/MyDrive/주피터_대피소/kaggle&github/Kaggle/참가중대회/otto~jan31,2023/train.parquet')
test = pl.read_parquet('/content/drive/MyDrive/주피터_대피소/kaggle&github/Kaggle/참가중대회/otto~jan31,2023/test.parquet')

In [ ]:
# train = train[:10_000_000]
train = train[:5000]

In [ ]:
sentences_df = pl.concat([train, test]).groupby('session').agg(
    pl.col('aid').alias('sentence'))

In [ ]:
sentences = sentences_df['sentence'].to_list()

In [ ]:
sentences[:5]

[[414639, 414639, 355818, 1421489],
 [297129],
 [1087638, 705234],
 [63658],
 [754412]]

In [ ]:
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import hashlib
import os
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
os.environ["PYTHONHASHSEED"] = str(42)
def hashfxn(x):
    return int(hashlib.md5(str(x).encode()).hexdigest(), 16)

In [ ]:
w2vec =Word2Vec(sentences=sentences,size=64, window=4, min_count=1, sg=0, workers=-1, seed=42, hashfxn=hashfxn)

TypeError: ignored

In [ ]:
w2vec.save("word2vec.model")

In [ ]:
model = Word2Vec.load("word2vec.model")

In [ ]:
%%time

from annoy import AnnoyIndex

aid2idx = {aid: i for i, aid in enumerate(w2vec.wv.index_to_key)}
index = AnnoyIndex(64, 'euclidean')

for aid, idx in aid2idx.items():
    index.add_item(idx, w2vec.wv.vectors[idx])

index.build(10,n_jobs=-1)

In [ ]:
from collections import defaultdict
import pandas as pd
import numpy as np
session_types = ['clicks', 'carts', 'orders']
test_session_AIDs = test.to_pandas().reset_index(drop=True).groupby('session')['aid'].apply(list)
test_session_types = test.to_pandas().reset_index(drop=True).groupby('session')['type'].apply(list)

In [ ]:
labels = []
type_weight_multipliers = {0: 1, 1: 6, 2: 3}

for AIDs, types in zip(test_session_AIDs,test_session_types):
#     print(AIDs,types)
    if len(AIDs) >= 20:
        # if we have enough aids (over equals 20) we don't need to look for candidates! we just use the old logic
        weights=np.logspace(0.1,1,len(AIDs),base=2, endpoint=True)-1
        aids_temp=defaultdict(lambda: 0)
        for aid,w,t in zip(AIDs,weights,types):
            aids_temp[aid]+= w * type_weight_multipliers[t]

        sorted_aids=[k for k, v in sorted(aids_temp.items(), key=lambda item: -item[1])]
        labels.append(sorted_aids[:20])
    else:
        # here we don't have 20 aids to output -- we will use word2vec embeddings to generate candidates!
        AIDs = list(dict.fromkeys(AIDs[::-1]))
        # let's grab the most recent aid
        most_recent_aid = AIDs[0]
#         print(most_recent_aid)
        # and look for some neighbors!
        nns = [w2vec.wv.index_to_key[i] for i in index.get_nns_by_item(aid2idx[most_recent_aid], 21)[1:]]

        labels.append((AIDs+nns)[:20])

In [ ]:
cnt =0
for i in range(len(labels)) :
    if len(labels[i]) < 20 :
        cnt+=1

(cnt /len(labels) )*100

In [ ]:
session_types = ['clicks','carts','orders']
labels_as_strings = [' '.join([str(l) for l in lls]) for lls in labels]

predictions = pd.DataFrame(data={'session_type': test_session_AIDs.index, 'labels': labels_as_strings, 's_type' : 0})

prediction_dfs = []

type2id = {'clicks' :0, "carts" : 1, 'orders' : 2}


for st in session_types:
    prediction = predictions.copy()
    prediction.session_type = prediction.session_type.astype('str') + f'_{st}'
    prediction.s_type = st
    prediction.s_type = prediction.s_type.apply(lambda x : type2id[x])
    prediction_dfs.append(prediction)

w2v_submission = pd.concat(prediction_dfs).reset_index(drop=True)
w2v_submission

In [ ]:
w2v_submission = w2v_submission.sort_values(['session_type', 's_type'], ascending=True)
w2v_submission

In [ ]:
%%time
train_pairs = (pl.concat([train, test])
    .groupby('session').agg([
        pl.col('aid'),
        pl.col('aid').shift(-1).alias('aid_next')
    ])
    .explode(['aid', 'aid_next'])
    .drop_nulls()
)[['aid', 'aid_next']]

In [ ]:
cardinality_aids = max(train_pairs['aid'].max(),train_pairs['aid_next'].max())
cardinality_aids

In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

class ClickDataset(Dataset) :
    def __init__(self,pairs) :
        self.aid1 = pairs['aid'].to_numpy()
        self.aid2 = pairs['aid_next'].to_numpy()
    def __getitem__(self, idx) :
        aid1 = self.aid1[idx]
        aid2 = self.aid2[idx]
        return [aid1,aid2]
    def __len__(self) :
        return len(self.aid1)

train_ds = ClickDataset(train_pairs[:len(train)])
test_ds = ClickDataset(train_pairs[len(train):])

In [ ]:
!pip install merlin-dataloader==0.0.2

In [ ]:
from merlin.loader.torch import Loader
from merlin.io import Dataset

In [ ]:
train_pairs[:len(train)].to_pandas().to_parquet('train_pairs.parquet')
train_pairs[len(train):].to_pandas().to_parquet('valid_pairs.parquet')

In [ ]:
train_ds = Dataset('train_pairs.parquet')
train_dl_merlin = Loader(train_ds, 65536, True)

In [ ]:
%%time

for batch, _ in train_dl_merlin:
    aid1, aid2 = batch['aid'], batch['aid_next']

In [ ]:
class MatrixFactorization(nn.Module) :
    def __init__(self, n_aids, n_factors) :
        super().__init__()
        self.aid_factors = nn.Embedding(n_aids, n_factors, sparse=True)

    def forward(self, aid1,aid2) :
        aid1 = self.aid_factors(aid1)
        aid2 = self.aid_factors(aid2)
        return (aid1*aid2).sum(dim=1)

class AverageMeter(object) :
    def __init__(self, name, fmt=':f') :
        self.name = name
        self.fmt =fmt
        self.reset()

    def reset(self) :
        self.avg=0
        self.val=0
        self.sum=0
        self.count = 0
    def update(self, val, n=1) :
        self.val = val
        self.sum +=val*n
        self.count +=n
        self.avg = self.sum / self.count

    def __str__(self) :
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

valid_ds = Dataset('valid_pairs.parquet')
valid_dl_merlin = Loader(valid_ds, 65536, True)

In [ ]:
from torch.optim import SparseAdam
import torch

num_epochs = 10
lr = 0.1

model = MatrixFactorization(cardinality_aids+1, 32)
optimizer = SparseAdam(model.parameters(), lr=lr)
criterion = nn.BCEWithLogitsLoss()

In [ ]:
%%time

for epoch in range(num_epochs):
    for batch, _ in train_dl_merlin:
        model.train().cuda()
        losses = AverageMeter('Loss', ':.4e')

        aid1, aid2 = batch['aid'], batch['aid_next']
        output_pos = model(aid1, aid2)
        output_neg = model(aid1, aid2[torch.randperm(aid2.shape[0])])

        output = torch.cat([output_pos, output_neg])
        targets = torch.cat([torch.ones_like(output_pos), torch.zeros_like(output_pos)])
        loss = criterion(output, targets)
        losses.update(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval()

    with torch.no_grad():
        accuracy = AverageMeter('accuracy')
        for batch, _ in valid_dl_merlin:
            aid1, aid2 = batch['aid'], batch['aid_next']
            output_pos = model(aid1, aid2)
            output_neg = model(aid1, aid2[torch.randperm(aid2.shape[0])])
            accuracy_batch = torch.cat([output_pos.sigmoid() > 0.5, output_neg.sigmoid() < 0.5]).float().mean()
            accuracy.update(accuracy_batch, aid1.shape[0])

    print(f'{epoch+1:02d}: * TrainLoss {losses.avg:.3f}  * Accuracy {accuracy.avg:.3f}')

In [ ]:
embeddings = model.aid_factors.weight.detach().cpu().numpy()

In [ ]:
%%time

from annoy import AnnoyIndex

fm = AnnoyIndex(32, 'euclidean')
for i, v in enumerate(embeddings) :
    fm.add_item(i,v)

fm.build(10)

In [ ]:
from collections import defaultdict

session_types = ['clicks', 'carts', 'orders']
test_session_AIDs = test.to_pandas().reset_index(drop=True).groupby('session')['aid'].apply(list)
test_session_types = test.to_pandas().reset_index(drop=True).groupby('session')['type'].apply(list)

labels = []


In [ ]:

type_weight_multipliers = {0: 1, 1: 6, 2: 3}
for AIDs, types in zip(test_session_AIDs, test_session_types):
    if len(AIDs) >= 20:
        # if we have enough aids (over equals 20) we don't need to look for candidates! we just use the old logic
        weights=np.logspace(0.1,1,len(AIDs),base=2, endpoint=True)-1
        aids_temp=defaultdict(lambda: 0)
        for aid,w,t in zip(AIDs,weights,types):
            aids_temp[aid]+= w * type_weight_multipliers[t]

        sorted_aids=[k for k, v in sorted(aids_temp.items(), key=lambda item: -item[1])]
        labels.append(sorted_aids[:20])
    else:
        # here we don't have 20 aids to output -- we will use approximate nearest neighbor search and our embeddings
        # to generate candidates!
        AIDs = list(dict.fromkeys(AIDs[::-1]))
        # let's grab the most recent aid
        most_recent_aid = AIDs[0]


        # and look for some neighbors!
        nns = fm.get_nns_by_item(most_recent_aid, 21)[1:]

        labels.append((AIDs+nns)[:20])

In [ ]:
session_types = ['clicks','carts','orders']
labels_as_strings = [' '.join([str(l) for l in lls]) for lls in labels]

predictions = pd.DataFrame(data={'session_type': test_session_AIDs.index, 'labels': labels_as_strings, 's_type' : 0})

prediction_dfs = []

type2id = {'clicks' : 0, 'carts' : 1, 'orders': 2}
for st in session_types:
    prediction = predictions.copy()
    prediction.session_type = prediction.session_type.astype('str') + f'_{st}'
    prediction.s_type = st
    prediction.s_type = prediction.s_type.apply(lambda x : type2id[x])
    prediction_dfs.append(prediction)

fm_submission = pd.concat(prediction_dfs).reset_index(drop=True)



In [ ]:
fm_submission = fm_submission.sort_values(['session_type', 's_type'], ascending=True)
fm_submission

# start here

In [ ]:
import pandas as pd

In [ ]:
fm_submission = pd.read_csv('/content/drive/MyDrive/주피터_대피소/kaggle&github/Kaggle/참가중대회/otto~jan31,2023/fm_submission1.csv')
w2v_submission = pd.read_csv('/content/drive/MyDrive/주피터_대피소/kaggle&github/Kaggle/참가중대회/otto~jan31,2023/w2v1_sub.csv')

In [ ]:
fm_submission

,session_type,labels,s_type
0,12899779_carts,59625 1389047 1711551 163940 1416435 943107 91...,1
1,12899779_clicks,59625 1389047 1711551 163940 1416435 943107 91...,0
2,12899779_orders,59625 1389047 1711551 163940 1416435 943107 91...,2
3,12899780_carts,1142000 736515 973453 582732 77422 215311 4487...,1
4,12899780_clicks,1142000 736515 973453 582732 77422 215311 4487...,0
...,...,...,...
5015404,14571580_clicks,202353 1154247 1176945 639737 600327 1845213 4...,0
5015405,14571580_orders,202353 1154247 1176945 639737 600327 1845213 4...,2
5015406,14571581_carts,1100210 1225597 1312252 1277199 1803829 118079...,1
5015407,14571581_clicks,1100210 1225597 1312252 1277199 1803829 118079...,0


In [ ]:
w2v_submission

,session_type,labels,s_type
0,12899779_carts,59625 1394590 595667 984259 1699996 1570951 12...,1
1,12899779_clicks,59625 1394590 595667 984259 1699996 1570951 12...,0
2,12899779_orders,59625 1394590 595667 984259 1699996 1570951 12...,2
3,12899780_carts,1142000 736515 973453 582732 324113 265263 362...,1
4,12899780_clicks,1142000 736515 973453 582732 324113 265263 362...,0
...,...,...,...
5015404,14571580_clicks,202353 42507 368686 238858 1522810 510980 1730...,0
5015405,14571580_orders,202353 42507 368686 238858 1522810 510980 1730...,2
5015406,14571581_carts,1100210 710104 196166 1515538 1478497 1357622 ...,1
5015407,14571581_clicks,1100210 710104 196166 1515538 1478497 1357622 ...,0


기준점이 될 sub

In [ ]:
sub = pd.merge(fm_submission, w2v_submission, on=['session_type', 'labels'], how='left')

In [ ]:
sub = sub[:2_000_000]

In [ ]:
sub

,session_type,labels,s_type_x,s_type_y
0,12899779_carts,59625 1389047 1711551 163940 1416435 943107 91...,1,NaN
1,12899779_clicks,59625 1389047 1711551 163940 1416435 943107 91...,0,NaN
2,12899779_orders,59625 1389047 1711551 163940 1416435 943107 91...,2,NaN
3,12899780_carts,1142000 736515 973453 582732 77422 215311 4487...,1,NaN
4,12899780_clicks,1142000 736515 973453 582732 77422 215311 4487...,0,NaN
...,...,...,...,...
1999995,13566444_carts,661295 1360305 186394 1236250 631479 1579175 1...,1,NaN
1999996,13566444_clicks,661295 1360305 186394 1236250 631479 1579175 1...,0,NaN
1999997,13566444_orders,661295 1360305 186394 1236250 631479 1579175 1...,2,NaN
1999998,13566445_carts,418247 216371 125278 1263754 691001 1464530 87...,1,NaN


w2v label

In [ ]:
w_labels = w2v_submission['labels']
w_labels

0          59625 1394590 595667 984259 1699996 1570951 12...
1          59625 1394590 595667 984259 1699996 1570951 12...
2          59625 1394590 595667 984259 1699996 1570951 12...
3          1142000 736515 973453 582732 324113 265263 362...
4          1142000 736515 973453 582732 324113 265263 362...
                                 ...                        
5015404    202353 42507 368686 238858 1522810 510980 1730...
5015405    202353 42507 368686 238858 1522810 510980 1730...
5015406    1100210 710104 196166 1515538 1478497 1357622 ...
5015407    1100210 710104 196166 1515538 1478497 1357622 ...
5015408    1100210 710104 196166 1515538 1478497 1357622 ...
Name: labels, Length: 5015409, dtype: object

In [ ]:
w_labels1 = w_labels[:2000000]



In [ ]:
w_labels1 = w_labels1.str.split(' ')

In [ ]:
w_labels1

0          [59625, 1394590, 595667, 984259, 1699996, 1570...
1          [59625, 1394590, 595667, 984259, 1699996, 1570...
2          [59625, 1394590, 595667, 984259, 1699996, 1570...
3          [1142000, 736515, 973453, 582732, 324113, 2652...
4          [1142000, 736515, 973453, 582732, 324113, 2652...
                                 ...                        
1999995    [661295, 866639, 78490, 375184, 1060514, 84854...
1999996    [661295, 866639, 78490, 375184, 1060514, 84854...
1999997    [661295, 866639, 78490, 375184, 1060514, 84854...
1999998    [418247, 216371, 125278, 1699452, 570590, 1041...
1999999    [418247, 216371, 125278, 1699452, 570590, 1041...
Name: labels, Length: 2000000, dtype: object

fm label

In [ ]:
f_labels= fm_submission['labels']
f_labels

0          59625 1389047 1711551 163940 1416435 943107 91...
1          59625 1389047 1711551 163940 1416435 943107 91...
2          59625 1389047 1711551 163940 1416435 943107 91...
3          1142000 736515 973453 582732 77422 215311 4487...
4          1142000 736515 973453 582732 77422 215311 4487...
                                 ...                        
5015404    202353 1154247 1176945 639737 600327 1845213 4...
5015405    202353 1154247 1176945 639737 600327 1845213 4...
5015406    1100210 1225597 1312252 1277199 1803829 118079...
5015407    1100210 1225597 1312252 1277199 1803829 118079...
5015408    1100210 1225597 1312252 1277199 1803829 118079...
Name: labels, Length: 5015409, dtype: object

In [ ]:
f_labels[0]

'59625 1389047 1711551 163940 1416435 943107 912753 1326087 91232 1280423 833203 1302755 79854 1423983 195906 236066 1746464 811179 130146 1525206'

In [ ]:
f_labels1 = f_labels[:2000000]

In [ ]:
f_labels1 =f_labels1.str.split(' ')

In [ ]:
sub_lists1 = []
for i in range(len(f_labels1)) :
    sub_list = [item for sublist1 in zip(w_labels1[i], f_labels1[i]) for item in sublist1]
    sub_list = list(dict.fromkeys(sub_list))
    sub_lists1.append(sub_list[:20])
sub_lists1[:10]

[['59625',
  '1394590',
  '1389047',
  '595667',
  '1711551',
  '984259',
  '163940',
  '1699996',
  '1416435',
  '1570951',
  '943107',
  '1242449',
  '912753',
  '1244381',
  '1326087',
  '1398094',
  '91232',
  '1599644',
  '1280423',
  '1417080'],
 ['59625',
  '1394590',
  '1389047',
  '595667',
  '1711551',
  '984259',
  '163940',
  '1699996',
  '1416435',
  '1570951',
  '943107',
  '1242449',
  '912753',
  '1244381',
  '1326087',
  '1398094',
  '91232',
  '1599644',
  '1280423',
  '1417080'],
 ['59625',
  '1394590',
  '1389047',
  '595667',
  '1711551',
  '984259',
  '163940',
  '1699996',
  '1416435',
  '1570951',
  '943107',
  '1242449',
  '912753',
  '1244381',
  '1326087',
  '1398094',
  '91232',
  '1599644',
  '1280423',
  '1417080'],
 ['1142000',
  '736515',
  '973453',
  '582732',
  '324113',
  '77422',
  '265263',
  '215311',
  '362907',
  '448755',
  '53683',
  '1383529',
  '354201',
  '19703',
  '1615986',
  '735729',
  '1130126',
  '1028575',
  '475457',
  '1796048'],


In [ ]:
len(sub_lists1)

2000000

In [ ]:
labels_as_strings = [' '.join([str(l) for l in lls]) for lls in sub_lists1]
submission = pd.DataFrame(data={'session_type': sub['session_type'], 'labels': labels_as_strings})

submission = submission.sort_values('session_type', axis=0, ascending=True)


In [ ]:
submission.to_csv('/content/drive/MyDrive/주피터_대피소/kaggle&github/Kaggle/참가중대회/otto~jan31,2023/submission1.csv', index=False)

In [ ]:
submission

,session_type,labels
0,12899779_carts,59625 1394590 1389047 595667 1711551 984259 16...
1,12899779_clicks,59625 1394590 1389047 595667 1711551 984259 16...
2,12899779_orders,59625 1394590 1389047 595667 1711551 984259 16...
3,12899780_carts,1142000 736515 973453 582732 324113 77422 2652...
4,12899780_clicks,1142000 736515 973453 582732 324113 77422 2652...
...,...,...
1999995,13566444_carts,661295 866639 1360305 78490 186394 375184 1236...
1999996,13566444_clicks,661295 866639 1360305 78490 186394 375184 1236...
1999997,13566444_orders,661295 866639 1360305 78490 186394 375184 1236...
1999998,13566445_carts,418247 216371 125278 1699452 1263754 570590 69...
